In [ ]:
!unzip /content/drive/MyDrive/Segmentation/dataSet/ph2dataset.zip
!rm -r /content/trainy
!rm -r /content/trainx
!rm -r /content/sample_data

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam

In [ ]:
H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_data(dataset_path, split=0.2):
    images = sorted(glob(os.path.join(dataset_path, "image", "*.bmp")))
    masks = sorted(glob(os.path.join(dataset_path, "mask", "*.bmp")))
    #print(type(images))
    test_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=test_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=test_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [ ]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x                                ## (256, 256, 3)

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)                    ## (256, 256)
    x = np.expand_dims(x, axis=-1)              ## (256, 256, 1)
    x = np.concatenate([x, x], axis=-1)         ## (256, 256, 2)
    return x

In [ ]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 2])
    return x, y

def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [ ]:
# augmenting data
import cv2
from glob import glob
from albumentations import CenterCrop, RandomRotate90, HorizontalFlip, VerticalFlip
from tqdm import tqdm

def augment(images,masks,save_path,augment_data=True):
  H=256
  W=256
  for x,y in tqdm(zip(images,masks),total=len(images)):
    name=x.split('/')[-1].split(".")
    img_name=name[0]
    img_ext=name[1]

    name=x.split('/')[-1].split(".")
    mask_name=name[0]
    mask_ext=name[1]

    x=cv2.imread(x,cv2.IMREAD_COLOR)
    y=cv2.imread(y,cv2.IMREAD_COLOR)
    #print(x.shape)
    #print(y.shape)

    if augment_data==True:
      # Randome rotate
      aug=RandomRotate90(p=0.8)
      augmented=aug(image=x,mask=y)
      x2=augmented['image']
      y2=augmented['mask']
      # vertical Flip
      aug=VerticalFlip(p=0.6)
      augmented=aug(image=x,mask=y)
      x3=augmented['image']
      y3=augmented['mask']
      # Horizontal Flip Flip
      aug=HorizontalFlip(p=0.7)
      augmented=aug(image=x,mask=y)
      x4=augmented['image']
      y4=augmented['mask']
      #Save file
      save_images=[x,x2,x3,x4]
      save_masks=[y,y2,y3,y4]
    else:
      save_images=[x]
      save_masks=[y]
    idx=0
    for i,m in zip(save_images,save_masks):
      i=cv2.resize(i,(W,H))
      m=cv2.resize(m,(W,H))
      tmp_img_name=f"{img_name}_{idx}.{img_ext}"
      tmp_mask_name=f"{mask_name}_{idx}.{mask_ext}"
      image_path=os.path.join(save_path,"image",tmp_img_name)
      mask_path=os.path.join(save_path,"mask",tmp_mask_name)
      cv2.imwrite(image_path,i)
      cv2.imwrite(mask_path,m)
      idx+=1

In [ ]:
create_dir("augDataset")
create_dir("augDataset/image")
create_dir("augDataset/mask")

In [ ]:
path='/content/ph2_dataset'
images=sorted(glob(os.path.join(path,"trainx/*")))
masks=sorted(glob(os.path.join(path,"trainy/*")))
print(f"Orignal Image: {len(images)}")
print(f"Orignal Masks: {len(masks)}")

Orignal Image: 200
Orignal Masks: 200


In [ ]:
augment(images,masks,'/content/augDataset',True)

100%|██████████| 200/200 [00:04<00:00, 42.86it/s]


In [ ]:
path='/content/augDataset'
images=sorted(glob(os.path.join(path,"image/*")))
masks=sorted(glob(os.path.join(path,"mask/*")))
print(f"Augmented Image: {len(images)}")
print(f"Augmented Masks: {len(masks)}")

Augmented Image: 800
Augmented Masks: 800


In [ ]:
""" Folder For saving result"""
create_dir("Files")

In [ ]:
# seeding 

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
""" Hyperparameters """
batch_size = 16
lr = 1e-4 ## (0.0001)
num_epoch = 50

In [ ]:
""" Dataset : 60/20/20 """
dataset_path = "/content/augDataset"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

In [ ]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 480 - 480
Valid: 160 - 160
Test: 160 - 160


In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

In [ ]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 2), dtype=tf.float32, name=None))>

In [ ]:
train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

In [ ]:
if len(train_x) % batch_size != 0:
  train_steps += 1

if len(valid_x) % batch_size != 0:
  valid_steps += 1

**Double Unet Model Code**

In [ ]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply, AveragePooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19

def squeeze_excite_block(inputs, ratio=8):
    init = inputs       ## (b, 128, 128, 32)
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)     ## (b, 32)   -> (b, 1, 1, 32)
    se = Reshape(se_shape)(se)
    se = Dense(filters//ratio, activation="relu", use_bias=False)(se)
    se = Dense(filters, activation="sigmoid", use_bias=False)(se)

    x = Multiply()([inputs, se])
    return x

def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def conv_block(x, filters):
    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = squeeze_excite_block(x)

    return x

def encoder1(inputs):
    skip_connections = []

    model = VGG19(include_top=False, weights="imagenet", input_tensor=inputs)
    names = ["block1_conv2", "block2_conv2", "block3_conv4", "block4_conv4"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("block5_conv4").output
    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [256, 128, 64, 32]
    skip_connections.reverse()

    x = inputs
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation="bilinear")(x)
        x = Concatenate()([x, skip_connections[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, 1, padding="same")(inputs)
    x = Activation("sigmoid")(x)
    return x

def encoder2(inputs):
    num_filters = [32, 64, 128, 256]
    skip_connections = []

    x = inputs
    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPool2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 32]
    skip_2.reverse()

    x = inputs
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation="bilinear")(x)
        x = Concatenate()([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

def build_model(input_shape):
    inputs = Input(input_shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    output1 = output_block(x)

    x = inputs * output1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    output2 = output_block(x)
    outputs = Concatenate()([output1, output2])
    model = Model(inputs, outputs)
    return model


input_shape = (256, 256, 3)
model = build_model(input_shape)

80150528/80134624 [==============================] - 1s 0us/step


In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
  return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
## compiling model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
metrics = [dice_coef, iou, Recall(), Precision()]
model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 256, 256, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [ ]:
model_path="/content/Files/model.h5"
csv_path="/content/Files/data.csv"
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
]

In [ ]:
model.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 0.5268 - dice_coef: 0.5254 - iou: 0.3609 - recall: 0.8537 - precision: 0.5960
Epoch 1: val_loss improved from inf to 0.52919, saving model to /content/Files/model.h5
30/30 [==============================] - 125s 3s/step - loss: 0.5268 - dice_coef: 0.5254 - iou: 0.3609 - recall: 0.8537 - precision: 0.5960 - val_loss: 0.5292 - val_dice_coef: 0.4165 - val_iou: 0.2633 - val_recall: 0.2757 - val_precision: 0.9744 - lr: 1.0000e-04
Epoch 2/50
30/30 [==============================] - ETA: 0s - loss: 0.3449 - dice_coef: 0.6566 - iou: 0.4905 - recall: 0.9070 - precision: 0.8646
Epoch 2: val_loss improved from 0.52919 to 0.39884, saving model to /content/Files/model.h5
30/30 [==============================] - 75s 3s/step - loss: 0.3449 - dice_coef: 0.6566 - iou: 0.4905 - recall: 0.9070 - precision: 0.8646 - val_loss: 0.3988 - val_dice_coef: 0.5394 - val_iou: 0.3697 - val_recall: 0.6152 - val_precision: 0.9720 - lr: 1.0000e-04
Epo

In [ ]:
model.save('/content/drive/MyDrive/Segmentation/PH2Models/PH2_double_Unet.hdf5')

In [ ]:
create_dir("results")

In [ ]:
def read_test_image(x):
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = np.clip(image - np.median(image)+127, 0, 255)
    image = image/255.0
    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=0)
    return image                              ## (1, 256, 256, 3)


def read_test_mask(y):
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = mask.astype(np.float32)
    mask = mask/255.0
    mask = np.expand_dims(mask, axis=-1)
    return mask
def mask_to_3d(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

def parse(y_pred):
    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = y_pred[..., -1]
    y_pred = y_pred.astype(np.float32)
    y_pred = np.expand_dims(y_pred, axis=-1)
    return y_pred

In [ ]:
def evaluate_normal(model, x_data, y_data):
    THRESHOLD = 0.5
    total = []
    for i, (x, y) in tqdm(enumerate(zip(x_data, y_data)), total=len(x_data)):
        x = read_test_image(x)
        y = read_test_mask(y)
        _, h, w, _ = x.shape

        y_pred1 = parse(model.predict(x)[0][..., -2]>0.5)
        y_pred2 = parse(model.predict(x)[0][..., -1]>0.5)
        
        line = np.ones((h, 10, 3)) * 255.0
        
        all_images = [
            x[0] * 255.0, line,
            mask_to_3d(y) * 255.0, line,
            mask_to_3d(y_pred1) * 255.0, line,
            mask_to_3d(y_pred2) * 255.0
        ]
        mask = np.concatenate(all_images, axis=1)

        cv2.imwrite(f"results/{i}.png", mask)

In [ ]:
evaluate_normal(model, test_x, test_y)

100%|██████████| 160/160 [01:05<00:00,  2.43it/s]


In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
model.load_weights('/content/drive/MyDrive/Segmentation/PH2Models/PH2_double_Unet.hdf5')

In [ ]:
test_dataset=tf_dataset(test_x, test_y, batch_size)
test_steps = len(test_x)//batch_size

if len(test_x) % batch_size != 0:
  test_steps += 1
test_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 2), dtype=tf.float32, name=None))>

In [ ]:
res=model.evaluate(train_dataset, steps=test_steps)
print('\n-------------On Train  Set--------------------------\n')
print('________________________')
print('Dice Coef:      |   {:.2f}  |'.format(res[1]*100))
print('IoU:            |   {:.2f}  |'.format(res[2]*100))
print('Recall:         |   {:.2f}  |'.format(res[3]*100))
print('Precision:      |   {:.2f}  |'.format(res[4]*100))
print('________________________')
res=model.evaluate(test_dataset, steps=test_steps)
print('\n-------------On Test  Set--------------------------\n')
print('________________________')
print('Dice Coef:      |   {:.2f}  |'.format(res[1]*100))
print('IoU:            |   {:.2f}  |'.format(res[2]*100))
print('Recall:         |   {:.2f}  |'.format(res[3]*100))
print('Precision:      |   {:.2f}  |'.format(res[4]*100))
print('________________________')
res=model.evaluate(valid_dataset, steps=test_steps)
print('\n-------------On valid  Set--------------------------\n')
print('________________________')
print('Dice Coef:      |   {:.2f}  |'.format(res[1]*100))
print('IoU:            |   {:.2f}  |'.format(res[2]*100))
print('Recall:         |   {:.2f}  |'.format(res[3]*100))
print('Precision:      |   {:.2f}  |'.format(res[4]*100))
print('________________________')

10/10 [==============================] - 7s 673ms/step - loss: 0.0700 - dice_coef: 0.9088 - iou: 0.8333 - recall: 0.9840 - precision: 0.9769

-------------On Train  Set--------------------------

________________________
Dice Coef:      |   90.88  |
IoU:            |   83.33  |
Recall:         |   98.40  |
Precision:      |   97.69  |
________________________
10/10 [==============================] - 7s 668ms/step - loss: 0.0811 - dice_coef: 0.9066 - iou: 0.8296 - recall: 0.9753 - precision: 0.9714

-------------On Test  Set--------------------------

________________________
Dice Coef:      |   90.66  |
IoU:            |   82.96  |
Recall:         |   97.53  |
Precision:      |   97.14  |
________________________
10/10 [==============================] - 7s 672ms/step - loss: 0.0823 - dice_coef: 0.9043 - iou: 0.8256 - recall: 0.9720 - precision: 0.9718

-------------On valid  Set--------------------------

________________________
Dice Coef:      |   90.43  |
IoU:            |   82.56  